In [61]:
import import_ipynb
import Masterthesis_Persons_Statistics

import os
import re
import json
import random
import time
import csv
import urllib.parse
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


def statistics_persons(link, driver, FounderOrEmployee="E"):
    #open company link for extracting persons statistics
    driver.get(link)
    
    time.sleep(random.uniform(2, 5))
    #clicking "Persons" button
    try:
        link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "People"))  # Example XPATH"
        )
        
        # Click the button
        link.click()

    except Exception as e:
        print("error")

    time.sleep(random.uniform(2, 5))
    #clicking "show more" button
    try:
        # Wait until the button is clickable using XPath
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'org-people__show-more-button')]"))
        )
        
        # Click the button
        button.click()
    
    except Exception as e:
        print("An error occurred:", e)

    #Filter for founders if necessary
    if FounderOrEmployee=="F":
        try:
            searchinput = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID,"people-search-keywords"))
            )
            searchinput.clear()#making sure that there is not text inside because the sended key will be appended
            searchinput.send_keys("Founder")
            searchinput.send_keys(Keys.ENTER)
        except Exception as e:
            print("error")
    
        time.sleep(2)
    
    #df_1, df_2
    #Extract the first df of a company Persons statistics
    def firstdf(div_text):
        extracted_text = div_text 
        # Step 1: Split the text into lines and filter out empty lines
        lines = [line.strip() for line in extracted_text.split('\n') if line.strip()]
        
        # Step 2: Find the index of the first occurrence of "Hinzufügen" and capture the word before it
        hinzufuegen_count = 0
        before_hinzufuegen = None
        end_index = None
        
        for i, line in enumerate(lines):
            if "Add" in line:
                hinzufuegen_count += 1
                if hinzufuegen_count == 1:  # Capture the word before the first occurrence
                    before_hinzufuegen = lines[i - 1]
                if hinzufuegen_count == 2:  # Find the end index for extraction
                    end_index = i
                    break
        
        # Step 3: Extract lines up to the second "Hinzufügen"
        if end_index is not None:
            relevant_lines = lines[:end_index - 1]  # Exclude the second "Hinzufügen"
        else:
            relevant_lines = lines  # If not found, take all lines
        
        # Step 4: Remove unwanted words ("ausschalten") and format the data
        cleaned_lines = [re.sub(r'\bausschalten\b', '', line).strip() for line in relevant_lines]
        cleaned_lines = [line for line in cleaned_lines if line]  # Remove empty lines
        
        # Initialize lists to hold amounts and locations
        amounts = []
        locations = []
        
        # Step 5: Extract amounts and locations from the cleaned lines
        for line in cleaned_lines:
            match = re.match(r'(\d+)\s*(.*)', line)
            if match:
                amounts.append(int(match.group(1)))  # Convert to integer
                locations.append(match.group(2).strip())  # Get the location
        
        # Step 6: Create a DataFrame with the desired column name
        df_1 = pd.DataFrame({
            'Amount': amounts,
            before_hinzufuegen: locations  # Use the word before "Hinzufügen" as the column name
        })
        
        return df_1
    
    
    
    
    #Extract the second df of a company Persons statistics
    def seconddf(div_text):
        extracted_text = div_text 
        # Step 1: Split the text into lines and filter out empty lines
        lines = [line.strip() for line in extracted_text.split('\n') if line.strip()]
    
        
        # Step 2: Find the index of the second occurrence of "Hinzufügen"
        hinzufuegen_count = 0
        start_index = None
        before_hinzufuegen = None
        
        for i, line in enumerate(lines):
            if "Add" in line:
                hinzufuegen_count += 1
                if hinzufuegen_count == 2:  # Find the second occurrence
                    start_index = i - 1  # Start one word before "Hinzufügen"
                    before_hinzufuegen = lines[i - 1]  # Store the word before "Hinzufügen"
                    break
        
        # Step 3: Extract lines starting from one before the second "Hinzufügen"
    
        if start_index is not None:
            relevant_lines = lines[start_index:]  # Get lines from the start index to the end
        else:
            relevant_lines = []  # If not found, take no lines
        
        # Step 4: Remove unwanted words ("ausschalten") and format the data
        cleaned_lines = [re.sub(r'\bausschalten\b', '', line).strip() for line in relevant_lines]
        cleaned_lines = [line for line in cleaned_lines if line]  # Remove empty lines
        
        # Initialize lists to hold amounts and locations for the second DataFrame
        amounts_2 = []
        locations_2 = []
        
        # Step 5: Extract amounts and locations from the cleaned lines
    
        for line in cleaned_lines:
            match = re.match(r'(\d+)\s*(.*)', line)
            if match:
                amounts_2.append(int(match.group(1)))  # Convert to integer
                locations_2.append(match.group(2).strip())  # Get the location
        
        # Step 6: Create the second DataFrame
        df_2 = pd.DataFrame({
            'Amount': amounts_2,
            before_hinzufuegen: locations_2  # Use the word before "Hinzufügen" as the column name
        })
        
        # Step 7: Drop rows where the second column has no values
        df_2 = df_2[df_2.iloc[:, 1] != '']  # Keep only rows where the second column is not empty
        
        return df_2
    
    
    # Wait for the <div> to be present and extract the text for df_1, df_2
    try:
        div_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-carousel__content'))  # Replace with your div's XPath or other selector
        )
        
        # Extract the text
        div_text = div_element.text
        #df=tabolizer(div_text)
    
        #print(div_text)
    except Exception as e:
        print("error")
    
    
    df_1 = firstdf(div_text)
    df_2 = seconddf(div_text)
    
    
    
    
    #df_3 to df_6
    #Extract from the third df of a company Persons statistics
    def dffrom3(div_text,df_2):
        extracted_text = div_text
        # Step 1: Split the text into lines and filter out empty lines
        lines = [line.strip() for line in extracted_text.split('\n') if line.strip()]
        
        # Step 2: Find the first occurrence of a number
        first_number_index = None
        for i, line in enumerate(lines):
            if re.match(r'^\d+', line):  # Check if the line starts with a number
                first_number_index = i
                break
        
    
        # Step 3: Extract relevant lines starting from the first number
        if first_number_index is not None:
            relevant_lines = lines[first_number_index:]  # Get lines from the first number to the end
        else:
            relevant_lines = []  # If no numbers found, take no lines
        
        # Step 4: Remove unwanted words ("ausschalten") and format the data
        cleaned_lines = [re.sub(r'\bausschalten\b', '', line).strip() for line in relevant_lines]
        cleaned_lines = [line for line in cleaned_lines if line]  # Remove empty lines
        
        # Initialize lists to hold amounts and fields for the new DataFrame
        amounts = []
        fields = []
    
        
        # Step 5: Extract amounts and fields from the cleaned lines
        for line in cleaned_lines:
            match = re.match(r'(\d+)\s*(.*)', line)  # Match the number and the text
            if match:
                amounts.append(int(match.group(1)))  # Convert to integer
                fields.append(match.group(2).strip())  # Get the field
        
        # Step 6: Create the new DataFrame
        df_new = pd.DataFrame({
            'Amount': amounts,
            'Field': fields  # Naming the second column "Field"
        })
        
        
    
        
        # Step 7: Subtract df_2 from df_new using column index
        df_3 = df_new[~df_new.iloc[:, 1].isin(df_2.iloc[:, 1])]  # Using index 1 for 'Field'
        
        # Step 8: Change the name of the right column based on the value in df_3
        if not df_3.empty:  # Check if df_3 is not empty
            number_to_match = df_3.iloc[0, 0]  # Get the number from the first row, first column (Amount)
            
            # Find the line above the first occurrence of the number in the original lines
            column_name = None
            for i, line in enumerate(lines):
                if re.match(rf'^{number_to_match}\s', line):  # Match the line with the number
                    if i > 0:  # Ensure there's a line above it
                        column_name = lines[i - 1]
                    break
            
            # Rename the second column in df_3
            if column_name:
                df_3.columns = ['Amount', column_name]
        
        return df_3
    
    
    # Loop through the function above to extract df_3 until df_6
    try:
        time.sleep(random.uniform(2, 5))
        # Wait until the button is clickable using XPath
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Next']"))
        )
        
        # Click the button
        button.click()
    except Exception as e:
            print("error")
    
    df_3 = pd.DataFrame()
    df_4 = pd.DataFrame()
    df_5 = pd.DataFrame()
    df_6 = pd.DataFrame()
    
    
    list = [df_3,df_4,df_5,df_6]
    memory = df_2
    
    dfs = []
    
    for df in enumerate(list):
        try:
            time.sleep(random.uniform(2, 5))
            # Wait until the button is clickable using XPath
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Next']"))
            )
            
            # Click the button
            button.click()
            
    
            div_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-carousel__content'))  # Replace with your div's XPath or other selector
            )
            
    
            # Extract the text
            div_text = div_element.text
            #df=tabolizer(div_text)
        
            #print(div_text)
        except Exception as e:
            print("error")
    
    
        
        df = dffrom3(div_text,memory)
        memory = df
        dfs.append(df)
        #print(df)
    
    
    df_3,df_4,df_5,df_6 = dfs
    return df_1, df_2, df_3, df_4, df_5, df_6

In [8]:
# Put data in Dictonary of Dataframes
def persons_statistics(internal_id, Company_LinkedIN_Link, dataframes_Persons_Statistics, driver):
    try:
        # Company name and URL
        #company_name = "52_hertz"
        
        # Get data for male and female statistics
        df_1, df_2, df_3, df_4, df_5, df_6 = statistics_persons(Company_LinkedIN_Link, driver, "E")
        #Fdf_1, Fdf_2, Fdf_3, Fdf_4, Fdf_5, Fdf_6 = statistics_persons(f"https://www.linkedin.com/company/{company_name.replace('_', '-')}", driver, "F")
        
        # Store DataFrames in the dictionary using the new format
        dataframes_Persons_Statistics[f"df_1_{internal_id}"] = df_1
        dataframes_Persons_Statistics[f"df_2_{internal_id}"] = df_2
        dataframes_Persons_Statistics[f"df_3_{internal_id}"] = df_3
        dataframes_Persons_Statistics[f"df_4_{internal_id}"] = df_4
        dataframes_Persons_Statistics[f"df_5_{internal_id}"] = df_5
        dataframes_Persons_Statistics[f"df_6_{internal_id}"] = df_6
        """
        dataframes_Persons_Statistics[f"Fdf_1_{company_name}"] = Fdf_1
        dataframes_Persons_Statistics[f"Fdf_2_{company_name}"] = Fdf_2
        dataframes_Persons_Statistics[f"Fdf_3_{company_name}"] = Fdf_3
        dataframes_Persons_Statistics[f"Fdf_4_{company_name}"] = Fdf_4
        dataframes_Persons_Statistics[f"Fdf_5_{company_name}"] = Fdf_5
        dataframes_Persons_Statistics[f"Fdf_6_{company_name}"] = Fdf_6
        """
        # Example of how to access the dynamically created DataFrames from the dictionary
        #print(dataframes_Persons_Statistics[f"df_1_52_hertz"][f"df_1_{company_name}"])  # Accessing the employee DataFrame
        #print(dataframes_Persons_Statistics[f"df_1_52_hertz"][f"Fdf_1_{company_name}"])  # Accessing the founder DataFrame
        #dataframes_Persons_Statistics[f"df_1_52_hertz"][f"df_1_52_hertz"]
    except:
        pass
    return dataframes_Persons_Statistics
